In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

spark = SparkSession \
    .builder \
    .getOrCreate()

sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble


23/05/25 15:40:47 WARN Utils: Your hostname, DESKTOP-0KANSSR resolves to a loopback address: 127.0.1.1; using 172.25.123.83 instead (on interface eth0)
23/05/25 15:40:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/25 15:40:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Joining Datasets<br>
- Join
- leftOuterJoin
- rightOuterJoin
- fullOuterJoin

#### join: join(other, numPartitions=None)</br>
When called on datasets og type (K,V), and (K,W), returns a dataset of (K, (V,W)) pairs with all pairs of elementos for each key.


Creating RDD

In [2]:
ord = sc.textFile('/home/phillipefs/spark_dev/pyspark-end-to-end-developer/0 - PracticeFiles/Orders')
ord_items = sc.textFile('/home/phillipefs/spark_dev/pyspark-end-to-end-developer/0 - PracticeFiles/Order_items')

Find the subtotal for each ORDER_CUSTOMER_ID

In [3]:
# Get order_id and customer_id
ord_map = ord.map(lambda x : (x.split(',')[0], x.split(',')[2]))
ord_map.take(5)

[('1', '11599'), ('2', '256'), ('3', '12111'), ('4', '8827'), ('5', '11318')]

In [4]:
# Get order_id and value
itens_map = ord_items.map(lambda x : (x.split(',')[0], x.split(',')[4]))
itens_map.take(3)

[('1', '299.98'), ('2', '199.99'), ('3', '250.0')]

In [5]:
#Join
ord_subtotal = ord_map.join(itens_map)
ord_subtotal.take(10)

[('4', ('8827', '129.99')),
 ('10', ('5648', '299.95')),
 ('12', ('1837', '299.98')),
 ('16', ('7276', '79.95')),
 ('20', ('9198', '50.0')),
 ('24', ('11441', '199.99')),
 ('26', ('7562', '129.99')),
 ('40', ('12092', '399.98')),
 ('44', ('10500', '199.99')),
 ('50', ('5225', '79.96'))]

Converting to Dataframe

In [6]:
schema = StructType([
    StructField("id_order", StringType(), True), 
    StructField("customer_value",StructType([
        StructField("id_customer", StringType(), True),
        StructField("value", StringType(), True)
    ]))
])

In [7]:
ord_subtotal.toDF(schema=schema).show(5)

+--------+--------------+
|id_order|customer_value|
+--------+--------------+
|       4|{8827, 129.99}|
|      10|{5648, 299.95}|
|      12|{1837, 299.98}|
|      16| {7276, 79.95}|
|      20|  {9198, 50.0}|
+--------+--------------+
only showing top 5 rows



#### cogroup: cogroup(other, numPartitions=None)</br>
When called on datasets og type (K,V), and (K,W), returns a dataset of (k, (Interable V, Iterable W)) tuples

In [8]:
ord_map.cogroup(itens_map).map(lambda x : (x[0], tuple(map(list,x[1])))).take(5)

[('4', (['8827'], ['129.99'])),
 ('10', (['5648'], ['299.95'])),
 ('12', (['1837'], ['299.98'])),
 ('16', (['7276'], ['79.95'])),
 ('20', (['9198'], ['50.0']))]

#### cartesian: cartesian(other)</br>

In [9]:
rdd = sc.parallelize([1,2,3])
rdd.cartesian(rdd).collect()

[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]